In [10]:
import csv
import nltk.classify
import cPickle
import re

def get_category(attribute):
    if attribute == "Positive":
        return ["1", "0", "0"]
    elif attribute == "Negative":
        return ["0", "1", "0"]
    elif attribute == "Neutral":
        return ["0", "0", "1"]
    else:
        # raise ValueError('Attribute is wrong')
        print attribute
        raise ValueError('Attribute is wrong')

fpr = open('Sentiment.csv', 'rb')
reader = csv.reader(fpr)
next(reader, None)

['id',
 'candidate',
 'candidate_confidence',
 'relevant_yn',
 'relevant_yn_confidence',
 'sentiment',
 'sentiment_confidence',
 'subject_matter',
 'subject_matter_confidence',
 'candidate_gold',
 'name',
 'relevant_yn_gold',
 'retweet_count',
 'sentiment_gold',
 'subject_matter_gold',
 'text',
 'tweet_coord',
 'tweet_created',
 'tweet_id',
 'tweet_location',
 'user_timezone']

In [11]:
tweets = []
training_labels_y = []
training_vects_y = []
pos_count = 0
neg_count = 0
neu_count = 0
negation = 0
pos_tweets = []
pos_conf = []
neg_tweets = []
neg_conf = []
neu_tweets = []
neu_conf = []

for row in reader:
    if 'Trump' in row[1]: 
        tweet = row[15]
        if len(tweet) < 30:
            continue
        pattern = re.compile(
            r"(.*(never| no |nothing|nowhere|noone| non|none|nobody| not |\
                |havent|hasnt|hadnt|cant|couldnt|cannot|shouldnt|wasnt|\
                |wont|wouldnt|dont|doesnt|didnt|isnt|arent|\
                |n't|aint))([^.:;!?]*)([.:;!?])(.*)")
        m = re.search(pattern, tweet)
        if m:
            negation += 1
        label = row[5]
        confidence = row[6]
        if label == 'Positive':
            if tweet[0:2] == 'RT':
                tweet = tweet[3:]
            if tweet not in pos_tweets:
                pos_count += 1
                pos_tweets.append(tweet)
                pos_conf.append(confidence)
        if label == 'Negative':
            if tweet[0:2] == 'RT':
                tweet = tweet[3:]
            if tweet not in neg_tweets:
                neg_count += 1
                neg_tweets.append(tweet)
                neg_conf.append(confidence)
        if label == 'Neutral':
            if tweet[0:2] == 'RT':
                tweet = tweet[3:]
            if tweet not in neu_tweets:
                neu_count += 1
                neu_tweets.append(tweet)
                neu_conf.append(confidence)

        #training_vects_y.append(get_category(label))
        #training_labels_y.append(label)
#fpr.close()
print pos_count, neg_count, neu_count
print negation


368 1369 345
592


In [12]:
cPickle.dump(pos_tweets[0:345], open('pos_tweets.pkl','wb'))
cPickle.dump(pos_conf[0:345], open('pos_conf.pkl','wb'))
cPickle.dump(neg_tweets[0:690], open('neg_tweets.pkl','wb'))
cPickle.dump(neg_conf[0:690], open('neg_conf.pkl','wb'))
cPickle.dump(neu_tweets[0:345], open('neu_tweets.pkl','wb'))
cPickle.dump(neu_conf[0:345], open('neu_conf.pkl', 'wb'))

In [13]:
pos_tweets = pos_tweets[0:345]
neg_tweets = neg_tweets[0:690]
neu_tweets = neu_tweets[0:345]
pos_conf = pos_conf[0:345]
neg_conf = neg_conf[0:690]
neu_conf = neu_conf[0:345]

tweets = pos_tweets + neg_tweets + neu_tweets
labels = ['Positive'] * len(pos_tweets) + ['Negative'] * len(neg_tweets) + ['Neutral'] * len(neu_tweets)
conf = pos_conf + neg_conf + neu_conf
print len(tweets)
print len(labels)
print len(conf)

1380
1380
1380


In [14]:
from Preprocessor import Preprocessor
from FeatureExtractor import FeatureExtractor
import csv
import cPickle

In [15]:
preprocessor = Preprocessor('stopWords.txt')
feature_extractor = FeatureExtractor(usingNegation = False, usingStemming = False, usingCorrection = False)
feature_space = []
features_list = []
for tweet in tweets:
    processed_tweet = preprocessor.preprocess(tweet)
    features = feature_extractor.extract_features_training(processed_tweet)
    features_list.append(features)
    for feature in features:
        if feature not in feature_space:
            feature_space.append(feature)
print len(feature_space)

3198


In [16]:
cPickle.dump(feature_space, open('feature_space.pkl','wb'))
feature_extractor.get_feature_space('feature_space.pkl')
training_vects_x = []
for features in features_list:
    feature_vect = feature_extractor.build_feature_vector(features)
    training_vects_x.append(feature_vect)
print len(training_vects_x)


pos_vects = []
neg_vects = []
neu_vects = []
X = []
pos_conf = []
neg_conf = []
neu_conf = []

print len(conf)
for i in xrange(len(training_vects_x)):
    if training_vects_x[i] not in X:
        X.append(training_vects_x[i])
        label = labels[i]
        if label == 'Positive':
            pos_vects.append(training_vects_x[i])
            pos_conf.append(conf[i])
        elif label == 'Negative':
            neg_vects.append(training_vects_x[i])
            neg_conf.append(conf[i])
        else:
            neu_vects.append(training_vects_x[i])
            #print i
            neu_conf.append(conf[i])
            
#X_vects = X_vects_pos + X_vects_neg + X_vects_neu
Y = ['Positive'] * len(pos_vects) + ['Negative'] * len(neg_vects) + ['Neutral'] * len(neu_vects)
conf = pos_conf + neg_conf + neu_conf
print len(Y)
print len(X)
print len(conf)

1380
1380
1288
1288
1288


In [17]:
print len(pos_vects)
print len(neg_vects)
print len(neu_vects)

325
664
299


In [18]:
cPickle.dump(pos_vects, open('pos_vects.pkl','wb'))
cPickle.dump(pos_conf, open('pos_conf.pkl','wb'))
cPickle.dump(neg_vects, open('neg_vects.pkl','wb'))
cPickle.dump(neg_conf, open('neg_conf.pkl','wb'))
cPickle.dump(neu_vects, open('neu_vects.pkl','wb'))
cPickle.dump(neu_conf, open('neu_conf.pkl', 'wb'))